In [1]:
reset -fs

In [2]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import string
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('/Users/evan/msds_main/machine_learning_1/yelp/yelp_business_and_reviews',sep='\t',index_col=0)

/Users/evan/anaconda3/envs/ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,2,4,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = df.loc[:,['business_id', 'text', 'happy_hour']]
df['text'] = df['text'].astype(str)

,business_id,text,happy_hour
0,iCQpiavjjPzJ5_3gPD5Ebg,The pizza was okay. Not the best I've had. I p...,False
1,iCQpiavjjPzJ5_3gPD5Ebg,"came here on Monday, no line. its good, but no...",False
2,iCQpiavjjPzJ5_3gPD5Ebg,This place is truly a secret! Its so hidden t...,False
3,iCQpiavjjPzJ5_3gPD5Ebg,This SECRET PIZZA was a secret for about 5 st...,False
4,iCQpiavjjPzJ5_3gPD5Ebg,this is the BEST place to grab a quick slice o...,False


In [6]:
# Group data by business_id and combine review text
grouped = df.groupby(['business_id', 'happy_hour'])['text'].sum()
grouped_df = pd.DataFrame(grouped)

In [8]:
# Fix dataframe format and indexing
grouped = grouped_df.add_suffix('_aggregated').reset_index()

# Remove business_id (?)
data = grouped.loc[:,['happy_hour', 'text_aggregated']]

In [30]:
# Check number of companies and average length of review
print(len(data))
print(len(df)/len(data))

127.14063274117065

In [ ]:
data_short = data[:10000]

In [11]:
train_data, test_data, train_target, test_target = train_test_split(data_short['text_aggregated'],
                                                                    data_short['happy_hour'])

In [13]:
str_test_data = []
for text in test_data:
    str_test_data.append(str(text))

In [14]:
str_train_data = []
for text in train_data:
    str_train_data.append(str(text))

In [15]:
vectorizer = TfidfVectorizer(stop_words='english', binary=True, ngram_range=(1,2)) 
train_features = vectorizer.fit_transform(str_train_data)
test_features = vectorizer.transform(str_test_data)

In [16]:
model = LogisticRegression(C=1e9, solver='lbfgs', multi_class='multinomial', random_state=0)
model.fit(train_features, train_target)
predicted = model.predict(test_features)
    
accuracy = accuracy_score(predicted, test_target)

In [17]:
accuracy

0.9156

In [19]:
model2 = LinearSVC(fit_intercept=True, random_state=0)
model2.fit(train_features, train_target)
predicted2 = model2.predict(test_features)
    
accuracy2 = accuracy_score(predicted2, test_target)

0.916

In [ ]:
accuracy2

In [23]:
# check for unbalanced data
data_short.groupby('happy_hour').count()

,text_aggregated
happy_hour,
False,8702
True,1298


In [24]:
1298/8702

0.1491611123879568